<a href="https://colab.research.google.com/github/srivatsan88/Natural-Language-Processing/blob/master/Text_classification_Tensorflow_Multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Link to video explaining the code is availale at - https://youtu.be/dkpS2g4K08s

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import os
import datetime
import tensorflow_hub as hub
import numpy as np
    
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv('https://github.com/srivatsan88/YouTubeLI/blob/master/dataset/consumer_compliants.zip?raw=true', compression='zip', sep=',', quotechar='"')

In [ ]:
df.columns

Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID'],
      dtype='object')

In [ ]:
pd.set_option('display.max_colwidth', -1)
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,4/3/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Fraudulent loan,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Company has responded to the consumer and the CFPB and chooses not to provide a public response,TRUIST FINANCIAL CORPORATION,PA,None,None,Consent provided,Web,4/3/2020,Closed with explanation,Yes,NaN,3591341
1,3/12/2020,Debt collection,Payday loan debt,Attempts to collect debt not owed,Debt is not yours,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",None,CURO Intermediate Holdings,CO,806XX,None,Consent provided,Web,3/12/2020,Closed with explanation,Yes,NaN,3564184
2,2/6/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Credit denial,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times.",None,CAPITAL ONE FINANCIAL CORPORATION,OH,430XX,None,Consent provided,Web,2/6/2020,Closed with explanation,Yes,NaN,3521949
3,3/6/2020,Checking or savings account,Savings account,Managing an account,Banking errors,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter 

In [ ]:
X_train, X_test = train_test_split(df, test_size=0.2, random_state=111)

In [ ]:
from sklearn.utils import class_weight
class_weights = list(class_weight.compute_class_weight('balanced',
                                             np.unique(df['Product']),
                                             df['Product']))

In [ ]:
df['Product'].value_counts()

Debt collection                21772
Credit card or prepaid card    13193
Mortgage                       9799 
Checking or savings account    7003 
Student loan                   2950 
Vehicle loan or lease          2736 
Name: Product, dtype: int64

In [ ]:
class_weights.sort()

In [ ]:
class_weights

[0.43980801028844385,
 0.7258015614340938,
 0.9771915501581794,
 1.3673425674710837,
 3.2459322033898306,
 3.4998172514619883]

In [ ]:
weights={}

In [ ]:
for index, weight in enumerate(class_weights) :
  weights[index]=weight

In [ ]:
weights

{0: 0.43980801028844385,
 1: 0.7258015614340938,
 2: 0.9771915501581794,
 3: 1.3673425674710837,
 4: 3.2459322033898306,
 5: 3.4998172514619883}

In [ ]:
dataset_train = tf.data.Dataset.from_tensor_slices((X_train['Consumer complaint narrative'].values, X_train['Product'].values))
dataset_test = tf.data.Dataset.from_tensor_slices((X_test['Consumer complaint narrative'].values, X_test['Product'].values))

In [ ]:
for text, target in dataset_train.take(5):
  print ('Complaint: {}, Target: {}'.format(text, target))

Complaint: b"The below complaint was submitted to the CFPB numerous times prior, the Wells Fargo rep, XXXX XXXX, replies with the same general form response stating numerous attempts at resolution have been made and exhausted which is a lie. He also attempts to state he can not comment due to past litigation which is also a lie, he reverts to this reply so as to avoid detailing any supposed attempt at resolution which he can not as there is none. Further, he should be aware of resolution which is to refund fees totaling {$610.00} and has not done so, no refund received to date. Please see complaint below. The below complaint was submitted prior, a duplicate form response received from XXXX XXXX with Wells Fargo, one of several. Based on this, my complaint was not addressed. Further, a reply in XXXX was never received as was mentioned. XXXX replied by stating numerous attempts at resolution have been made but has not detailed one, there was no contact from anyone at Wells Fargo aside fr

In [ ]:
for text, target in dataset_test.take(5):
  print ('Complaint: {}, Target: {}'.format(text, target))

Complaint: b'I have a business checking account at BB & T. On XX/XX/2019, I attempted to deposit a check into my account and I received a message stating that I was over my monthly mobile deposit limit. I was confused because it was the first of the month and I had not deposited any checks since the previous month. I called BB & T and they said that I couldnt deposit checks into business accounts via the mobile app even though I had done that before. \n\nI was instructed to open a personal account, into which I could deposit checks via the mobile app. I was told that if I opened the account online I would have immediate access, that I could link my personal and business accounts, and immediately be able to transfer money between them. \n\nOn XX/XX/XXXX, I opened my personal account online. Though I successfully opened online, I did not have online access as I had been promised. Because I was traveling in an area where there were no BB & T branches, I could not go into a branch until XX

In [ ]:
table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant(['Debt collection', 'Credit card or prepaid card', 'Mortgage', 'Checking or savings account', 'Student loan', 'Vehicle loan or lease']),
        values=tf.constant([0, 1, 2, 3, 4, 5]),
    ),
    default_value=tf.constant(-1),
    name="target_encoding"
)

@tf.function
def target(x):
  return table.lookup(x)

In [ ]:
def show_batch(dataset, size=5):
  for batch, label in dataset.take(size):
      print(batch.numpy())
      print(target(label).numpy())

In [ ]:
show_batch(dataset_test,6)

b'I have a business checking account at BB & T. On XX/XX/2019, I attempted to deposit a check into my account and I received a message stating that I was over my monthly mobile deposit limit. I was confused because it was the first of the month and I had not deposited any checks since the previous month. I called BB & T and they said that I couldnt deposit checks into business accounts via the mobile app even though I had done that before. \n\nI was instructed to open a personal account, into which I could deposit checks via the mobile app. I was told that if I opened the account online I would have immediate access, that I could link my personal and business accounts, and immediately be able to transfer money between them. \n\nOn XX/XX/XXXX, I opened my personal account online. Though I successfully opened online, I did not have online access as I had been promised. Because I was traveling in an area where there were no BB & T branches, I could not go into a branch until XX/XX/2019. I

In [ ]:
def fetch(text, labels):
  return text, tf.one_hot(target(labels),6)

In [ ]:
train_data_f=dataset_train.map(fetch)
test_data_f=dataset_test.map(fetch)

In [ ]:
next(iter(train_data_f))

(<tf.Tensor: shape=(), dtype=string, numpy=b"The below complaint was submitted to the CFPB numerous times prior, the Wells Fargo rep, XXXX XXXX, replies with the same general form response stating numerous attempts at resolution have been made and exhausted which is a lie. He also attempts to state he can not comment due to past litigation which is also a lie, he reverts to this reply so as to avoid detailing any supposed attempt at resolution which he can not as there is none. Further, he should be aware of resolution which is to refund fees totaling {$610.00} and has not done so, no refund received to date. Please see complaint below. The below complaint was submitted prior, a duplicate form response received from XXXX XXXX with Wells Fargo, one of several. Based on this, my complaint was not addressed. Further, a reply in XXXX was never received as was mentioned. XXXX replied by stating numerous attempts at resolution have been made but has not detailed one, there was no contact fro

In [ ]:
train_data, train_labels = next(iter(train_data_f.batch(5)))
train_data, train_labels

(<tf.Tensor: shape=(5,), dtype=string, numpy=
 array([b"The below complaint was submitted to the CFPB numerous times prior, the Wells Fargo rep, XXXX XXXX, replies with the same general form response stating numerous attempts at resolution have been made and exhausted which is a lie. He also attempts to state he can not comment due to past litigation which is also a lie, he reverts to this reply so as to avoid detailing any supposed attempt at resolution which he can not as there is none. Further, he should be aware of resolution which is to refund fees totaling {$610.00} and has not done so, no refund received to date. Please see complaint below. The below complaint was submitted prior, a duplicate form response received from XXXX XXXX with Wells Fargo, one of several. Based on this, my complaint was not addressed. Further, a reply in XXXX was never received as was mentioned. XXXX replied by stating numerous attempts at resolution have been made but has not detailed one, there was no 

In [ ]:
embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"
hub_layer = hub.KerasLayer(embedding, output_shape=[128], input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_data[:1])

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[ 1.92570090e+00,  1.04540564e-01,  1.63910031e-01,
        -1.11070231e-01, -5.49944043e-02,  7.91307315e-02,
         4.86127660e-02,  2.75555283e-01, -8.30120146e-02,
         1.80027023e-01,  1.44885316e-01, -1.59288287e-01,
        -1.94997236e-01, -4.00722414e-01, -1.37285963e-01,
         3.61927778e-01, -3.39035988e-01, -3.47756073e-02,
        -6.16503179e-01,  1.44515026e+00,  3.19949985e-01,
         3.94565135e-01, -3.26071948e-01,  2.14843497e-01,
        -3.63146365e-02, -4.13899511e-01,  1.67138502e-01,
        -5.15444636e-01, -1.83536708e-01,  3.90481912e-02,
         9.09547061e-02, -2.18187377e-01,  1.16193175e-01,
        -2.13140637e-01,  2.62742490e-01,  3.77820075e-01,
        -2.58993626e-01, -5.08686364e-01, -2.61283755e-01,
        -5.75724877e-02,  5.14812469e-02, -1.78749681e-01,
        -8.78547728e-02, -5.78522384e-01,  3.56190205e-01,
         3.41897160e-01, -3.03279817e-01, -3.03550176e-02,
      

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
for units in [128, 128, 64 , 32]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))
  model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(6, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 128)               124642688 
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
train_data_f=train_data_f.shuffle(70000).batch(512)
test_data_f=test_data_f.batch(512)

In [ ]:
history = model.fit(train_data_f,
                    epochs=4,
                    validation_data=test_data_f,
                    verbose=1,
                    class_weight=weights)

Epoch 1/4
90/90 [==============================] - 9s 100ms/step - loss: 1.6476 - accuracy: 0.4482 - val_loss: 1.2779 - val_accuracy: 0.7771
Epoch 2/4
90/90 [==============================] - 9s 100ms/step - loss: 1.3280 - accuracy: 0.7920 - val_loss: 1.1924 - val_accuracy: 0.8509
Epoch 3/4
90/90 [==============================] - 9s 101ms/step - loss: 1.1963 - accuracy: 0.8554 - val_loss: 1.1780 - val_accuracy: 0.8651
Epoch 4/4
90/90 [==============================] - 9s 101ms/step - loss: 1.1556 - accuracy: 0.8848 - val_loss: 1.1696 - val_accuracy: 0.8739


In [ ]:
len(list(dataset_test))

11491

In [ ]:
results = model.evaluate(dataset_test.map(fetch).batch(11491), verbose=2)

print(results)

1/1 - 0s - loss: 1.1696 - accuracy: 0.8739
[1.1696072816848755, 0.8739013075828552]


In [ ]:
test_data, test_labels = next(iter(dataset_test.map(fetch).batch(45963)))

In [ ]:
y_pred=model.predict(test_data)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(test_labels.numpy().argmax(axis=1), y_pred.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.94      0.89      0.91      4295
           1       0.85      0.86      0.86      2583
           2       0.93      0.92      0.92      2015
           3       0.87      0.82      0.85      1461
           4       0.81      0.84      0.82       611
           5       0.55      0.84      0.66       526

    accuracy                           0.87     11491
   macro avg       0.82      0.86      0.84     11491
weighted avg       0.88      0.87      0.88     11491



Classification Report with no class weights assigned
           


```
              precision    recall  f1-score   support

           0       0.92      0.91      0.92      4295
           1       0.84      0.88      0.86      2583
           2       0.90      0.94      0.92      2015
           3       0.86      0.84      0.85      1461
           4       0.86      0.78      0.81       611
           5       0.69      0.62      0.65       526

    accuracy                           0.88     11491
   macro avg       0.85      0.83      0.84     11491
weighted avg       0.88      0.88      0.88     11491
```




In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_labels.numpy().argmax(axis=1), y_pred.argmax(axis=1))

array([[3815,  167,   76,   32,   80,  125],
       [ 125, 2222,   13,  109,   10,  104],
       [  37,   21, 1848,   23,   21,   65],
       [  37,  162,   30, 1204,    5,   23],
       [  18,   18,   11,    4,  511,   49],
       [  29,   21,   17,   10,    7,  442]])

Confusion matrix without weights assigned

```
array([[3910,  165,   80,   34,   50,   56],
       [ 128, 2274,   20,  128,    3,   30],
       [  36,   27, 1893,   18,   16,   25],
       [  41,  149,   29, 1227,    5,   10],
       [  41,   35,   30,    6,  474,   25],
       [  72,   50,   60,   14,    5,  325]])
```

